In [6]:
using StatsBase

file_input = []

path = "C:\\Users\\advai\\Desktop\\test1.txt"

open(path,"r") do x
        for i in eachline(x)
                # println(i)
                append!(file_input,split(i," "))
        end
end

k = parse(Int,file_input[1])
number_of_dna = parse(Int,file_input[2])
number_of_iterations = parse(Int,file_input[3])

dna_seq= file_input[4:end]



5-element Array{Any,1}:
 "AATTACTACTACC"
 "ATTACCTCACATT"
 "AATTACTACTACT"
 "ATTACTACTACTC"
 "CTTCATTACTTGG"

In [5]:
# Returns frequency profile
function make_profile(bmotif_temp,k,laplace=true)
        profile=Dict('a'=>zeros(k),'c'=>zeros(k),'g'=>zeros(k),'t'=>zeros(k))
        # println("list passed: $bmotif_temp")
        for i in bmotif_temp
                for j in 1:k
                        if  i[j]== 'A' || i[j]== 'a'
                                profile['a'][j]+=1
                        elseif i[j]== 'C' || i[j]== 'c'
                                profile['c'][j]+=1
                        elseif i[j]== 'G' || i[j]== 'g'
                                profile['g'][j]+=1
                        elseif i[j]== 'T' || i[j]== 't'
                                profile['t'][j]+=1
                        else
                                print("Wrong nucleotide input")
                        end
                end
        end
        # println("Profile is $profile")
        z_present = false
        if laplace == true
                for key = keys(profile)
                                for i in profile[key]

                                        if 0 in i
                                                z_present = true
                                        end
                                        break
                                end
                end
                # print(z_present)
        end
        # println("before laplace $profile
        if z_present
                        for key = keys(profile)
                                profile[key]+=1
                        end
                        normalizer = profile['a'][1]+profile['c'][1]+profile['t'][1] + profile['g'][1]
                        # println(profile)
                        # println(normalizer)
                        for key = keys(profile)
                                profile[key]/=normalizer
                        end

                        return profile

        else
                normalizer = profile['a'][1]+profile['c'][1]+profile['t'][1] + profile['g'][1]
                for key = keys(profile)
                        profile[key]/=normalizer
                end
                return profile
        end
end

make_profile (generic function with 2 methods)

In [ ]:
# random number generator pg94-99, the else in deterministic false is for testing purpose
function random_generator(dna,profile,k,deterministic=true)
        score=1
        # score_dict=Dict()
        highest_score_motif = []
        highest_score=0
        for iter in 1:length(dna)-k+1
                for i in 1:k
                        # print(dna[iter:iter+k-1][i])
                        score*=profile[lowercase(dna[iter:iter+k-1][i])][i]
                end
                # println("Score for $dna[iter:iter+k-1] is $score")
                # score_dict[iter]=score
                if deterministic
                        if score>highest_score
                                highest_score = score
                                highest_score_motif = dna[iter:iter+k-1]
                                # println("Highest score $score for $highest_score_motif")
                        # deterministic when score = highest_score
                        end
                        score=1
                else
                        if score>highest_score
                                highest_score = score
                                highest_score_motif = dna[iter:iter+k-1]
                        elseif score == highest_score
                                pick = sample(1:2,1)
                                if pick == 1
                                        highest_score_motif = dna[iter:iter+k-1]
                                # else
                                #         println("Retained previous motif")
                                end
                        end
                        score=1
                end
        end

        return highest_score_motif

end

In [3]:
function gibbssampler(dna_seq,k,number_of_dna,number_of_iterations)
        random_kmer_start_position = Dict()
        length_of_sequence = length(dna_seq[1])
        best_motif = []
        for i in 1:number_of_dna

                random_kmer_start_position[i]=sample(1:length_of_sequence-k+1,1)[1]
        end

        # println(random_kmer_start_position)

        for row_number = keys(random_kmer_start_position)
                start = random_kmer_start_position[row_number]
                push!(best_motif,String(dna_seq[row_number][start:start+k-1]))
        end
        println(best_motif)

# bad idea to use temp=best_motif , coz operation on temp will reflect on best_motif.
        for j in 1:number_of_iterations
                index_to_be_omitted = sample(1:number_of_dna,1)[1]
                # println(index_to_be_omitted)
                # println("Best motif $best_motif")
                temp = []
                append!(temp,best_motif)
                deleteat!(temp,index_to_be_omitted)
                # println(temp)
                profile=make_profile(temp,k)
                best_motif[index_to_be_omitted]=random_generator(dna_seq[index_to_be_omitted],profile,k,true)
        end
        return best_motif
end

# println("Number of DNA is $number_of_dna")
output = gibbssampler(dna_seq,k,number_of_dna,number_of_iterations)
println("Output is")
for nn in output
        println(nn)
end

# println(number_of_dna)
score=0
for i in 1:k
        hamming_distance = number_of_dna
        count_dic = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j in output
                count_dic[j[i]]+=1
        end
        hamming_distance-=maximum(values(count_dic))
        score += hamming_distance
end

println(score)

Any["TACT", "TCAC", "ACTA", "TTCA", "AATT"]
Output is
ATTA
ATTA
ATTA
ATTA
ATTA
0
